In [11]:
# Import libraries
import numpy as np
import time
import pandas as pd
import sys
from scipy.constants import mu_0
sys.path.insert(1, '../src')

# Load function that performs global search in lookup table
from EM1D import GlobalSearch_2Lay

In [12]:
def Q_from_Sigma(sigma, s, freq=9000, mu_0=mu_0):
    """ Function that back transforms Sigma_app to Quadrature values
    using the LIN approximation function 
    
    Parameters: 
    1. sigma: apparent conductivity [S/m]
    2. s: coil offset [m]
    
    Returns:
    Q : quadrature values
    """
    
    Q = sigma * (2 *np.pi * freq) * mu_0 * s**2 /4
    return Q

In [2]:
# Load lookup table and sampling 
LUT = np.load('data/LUTable_2Lay_Texel.npy')
conds = np.load('data/conds.npy')
thicks =  np.load('data/thicks.npy')


In [7]:
Dataframe = pd.read_csv('data/DualemTexel_27-03-2024.csv', 
                        names = ['Time','Altitude','Latitude','Longitude',
                                   'HDOP','Counter','Voltage','Current',
                                   'Pdeg','Rdeg','DualemTimei2','H2mS_m',
                                   'H2ppt','P2mS_m','P2ppt','DualemTimei4',
                                   'H4mS_m','H4ppt','P4mS_m','P4ppt',
                                   'DualemTimei8','H8mS_m','H8ppt','P8mS_m','P8ppt'])

In [8]:
Dataframe

,Time,Altitude,Latitude,Longitude,HDOP,Counter,Voltage,Current,Pdeg,Rdeg,...,DualemTimei4,H4mS_m,H4ppt,P4mS_m,P4ppt,DualemTimei8,H8mS_m,H8ppt,P8mS_m,P8ppt
0,12:13:11,-2.059,53.135728,4.872635,0.49,4312,10.92,13.2,-2.5,3.0,...,180715.446,179.9,43.27,166.9,26.89,180715.446,69.5,196.02,194.8,201.20
1,12:13:13,-2.060,53.135730,4.872653,0.49,4313,10.84,13.2,-0.9,1.9,...,180716.446,179.8,43.28,168.6,26.73,180716.446,68.7,194.25,194.4,197.63
2,12:13:14,-2.060,53.135732,4.872662,0.49,4314,10.92,13.2,-2.2,8.5,...,180717.447,180.8,43.45,172.8,26.88,180717.447,67.8,192.90,195.2,197.33
3,12:13:15,-2.043,53.135733,4.872665,0.49,4315,10.85,13.2,-4.9,4.4,...,180718.447,179.2,43.36,174.8,26.19,180718.447,63.5,188.40,194.6,188.78
4,12:13:15,-2.043,53.135733,4.872665,0.49,4316,10.93,13.2,2.1,9.3,...,180719.447,174.5,42.60,173.6,24.61,180719.447,59.9,180.17,189.4,171.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,12:11:03,-3.085,53.136419,4.874212,0.56,86,7.08,10.6,36.2,46.7,...,180507.865,201.8,52.44,191.2,31.76,180507.865,66.9,228.62,217.9,258.32
2211,12:11:04,-3.075,53.136419,4.874212,0.56,87,6.93,10.6,36.2,46.6,...,180508.865,202.1,52.34,192.4,31.33,180508.865,66.6,228.60,217.4,257.93
2212,12:11:05,-3.064,53.136419,4.874212,0.56,88,7.07,10.6,36.2,46.6,...,180509.866,202.2,52.53,194.0,31.58,180509.866,66.9,229.00,217.9,258.24
2213,12:11:06,-3.061,53.136419,4.874212,0.56,89,6.91,10.6,36.2,46.6,...,180510.866,202.1,52.65,192.4,31.70,180510.866,66.6,229.26,218.2,258.54


In [23]:
# Obtain Quadrature and In-Phase values

H2Q = Q_from_Sigma(Dataframe['H2mS_m']/1000, 2)
P2Q = Q_from_Sigma(Dataframe['P2mS_m']/1000, 2.1)
H4Q = Q_from_Sigma(Dataframe['H4mS_m']/1000, 4)
P4Q = Q_from_Sigma(Dataframe['P4mS_m']/1000, 4.1)
H8Q = Q_from_Sigma(Dataframe['H8mS_m']/1000, 8)
P8Q = Q_from_Sigma(Dataframe['P8mS_m']/1000, 8.1)

H2IP = Dataframe['H2ppt']/1000
P2IP = Dataframe['P2ppt']/1000
H4IP = Dataframe['H4ppt']/1000
P4IP = Dataframe['P4ppt']/1000
H8IP = Dataframe['H8ppt']/1000
P8IP = Dataframe['P8ppt']/1000

In [29]:
# Arrange data array for inversion

data = np.array(pd.concat([H2Q, H4Q, H8Q, H2IP, H4IP, H8IP, P2Q, P4Q, P8Q, P2IP, P4IP, P8IP], axis=1))

In [ ]:
# number of 1D models
npos = len(data)

# Estimate with both Quadrature and In Phase
model_est = [] # Empty array for estimated model

print('Starting global search ...')

starttime = time.time()
for p in range(npos):
    model_est.append(GlobalSearch_2Lay(LUT, data[p], conds, thicks, nsl=len(conds)))
endtime = time.time() - starttime

print('Global search Q+IP excution for ', npos, ' positions: ', f"{(endtime/60):.3}", 'minutes')

# Save estimated models
np.save('results/model_2Lay_GS_Texel', model_est)